In [1]:
import os
import rasterio as rio
import rasterio.windows as wnd
import rasterio.transform as transform
import matplotlib.pyplot as plt
import numpy as np

In [2]:
base_dir = "/home/dsa/DSA/qc"
man_dir = os.path.join(base_dir, "manual_labels")
full_scene_ref_dir = os.path.join(base_dir, "full_scene_ref_bands")

In [3]:
scene_ids = [i[:-8] for i in os.listdir(man_dir) if i[-3:] == "tif"]


In [4]:
def save_qc_chunk(man_out_path, qc_data):
    # convert cell values to 0,1 from 1,2 for image diffing
    qc_data = qc_data - np.ones(qc_data.shape)
    # save qc chunk as .npy file for convenience/eliminating rasterio dependency
    np.save(man_out_path, qc_data, allow_pickle=True)

In [5]:
def get_rounded_window(qc_bounds, full_scene_band_path):
    with rio.open(full_scene_band_path) as img_ref:
        image_meta = img_ref.meta.copy()
        image_transform = img_ref.transform
        
    window = wnd.from_bounds(qc_bounds[0],
                                qc_bounds[1],
                                qc_bounds[2],
                                qc_bounds[3],
                                transform=image_transform)
    return wnd.Window(round(window.col_off), round(window.row_off), int(window.width), int(window.height))

In [6]:
def get_qc_chunks(window, chunk_size):
    
    row = window.row_off / chunk_size
    col = window.col_off / chunk_size
    row_index = int((row - int(row)) * chunk_size)
    col_index = int((col - int(col)) * chunk_size)
    
    chunks_needed = []
    chunks_needed.append((int(row), int(col)))
    extra_row_needed = (chunk_size - row_index) < window.height
    extra_col_needed = (chunk_size - col_index) < window.width
    if extra_row_needed:
        chunks_needed.append((int(row) + 1, int(col)))
    if extra_col_needed:
        chunks_needed.append((int(row), int(col) + 1))
    if extra_row_needed and extra_col_needed:
        chunks_needed.append((int(row) + 1, int(col) + 1))
    
    return {"chunks":sorted(chunks_needed), "row": row_index, "col": col_index,
            "width": window.width, "height": window.height}

In [7]:
chunk_params = {}
for i in scene_ids:
    man_path = os.path.join(man_dir, i + "_man.tif")
    
    full_scene_band_path = os.path.join(full_scene_ref_dir, i, i + "_B1.TIF")
    man_out_path = os.path.join(man_dir, i + "_man.npy")
    if i == "ryder_bay_abj":
        full_scene_band_path = os.path.join(full_scene_ref_dir, "LC82201072015017LGN00", "LC82201072015017LGN00_B1.TIF")
        man_out_path = os.path.join(man_dir, "LC82201072015017LGN00_ryder_man.npy")
    assert os.path.exists(man_path)
    assert os.path.exists(full_scene_band_path)
    
    with rio.open(man_path) as qc_chunk:
        qc_bounds = qc_chunk.bounds
        qc_data = qc_chunk.read(1)
    save_qc_chunk(man_out_path, qc_data)
    rounded_window = get_rounded_window(qc_bounds, full_scene_band_path)
    qc_chunks = get_qc_chunks(rounded_window, 512)
    chunk_params[i] = qc_chunks

/home/dsa/DSA/qc/manual_labels/LC80311222014338LGN00_man.tif
/home/dsa/DSA/qc/manual_labels/LC80311222014338LGN00_man.npy
/home/dsa/DSA/qc/manual_labels/LC80631112014002LGN00_man.tif
/home/dsa/DSA/qc/manual_labels/LC80631112014002LGN00_man.npy
/home/dsa/DSA/qc/manual_labels/LC81041072013303LGN00_man.tif
/home/dsa/DSA/qc/manual_labels/LC81041072013303LGN00_man.npy
/home/dsa/DSA/qc/manual_labels/LC81431082013352LGN00_man.tif
/home/dsa/DSA/qc/manual_labels/LC81431082013352LGN00_man.npy
/home/dsa/DSA/qc/manual_labels/LC81681112014002LGN00_man.tif
/home/dsa/DSA/qc/manual_labels/LC81681112014002LGN00_man.npy
/home/dsa/DSA/qc/manual_labels/LC82071182013336LGN00_man.tif
/home/dsa/DSA/qc/manual_labels/LC82071182013336LGN00_man.npy
/home/dsa/DSA/qc/manual_labels/LC82091172014001LGN00_man.tif
/home/dsa/DSA/qc/manual_labels/LC82091172014001LGN00_man.npy
/home/dsa/DSA/qc/manual_labels/LC82131132013362LGN00_man.tif
/home/dsa/DSA/qc/manual_labels/LC82131132013362LGN00_man.npy
/home/dsa/DSA/qc/manual_

In [8]:
import json

In [9]:
print(chunk_params)

{'LC80311222014338LGN00': {'chunks': [(6, 1), (6, 2), (7, 1), (7, 2)], 'row': 317, 'col': 498, 'width': 333, 'height': 333}, 'LC80631112014002LGN00': {'chunks': [(6, 6), (6, 7)], 'row': 165, 'col': 272, 'width': 333, 'height': 333}, 'LC81041072013303LGN00': {'chunks': [(6, 14), (7, 14)], 'row': 282, 'col': 20, 'width': 333, 'height': 334}, 'LC81431082013352LGN00': {'chunks': [(3, 6), (4, 6)], 'row': 508, 'col': 64, 'width': 333, 'height': 334}, 'LC81681112014002LGN00': {'chunks': [(6, 2), (6, 3), (7, 2), (7, 3)], 'row': 391, 'col': 318, 'width': 334, 'height': 334}, 'LC82071182013336LGN00': {'chunks': [(7, 8), (7, 9), (8, 8), (8, 9)], 'row': 488, 'col': 482, 'width': 334, 'height': 334}, 'LC82091172014001LGN00': {'chunks': [(15, 6), (15, 7), (16, 6), (16, 7)], 'row': 296, 'col': 505, 'width': 333, 'height': 333}, 'LC82131132013362LGN00': {'chunks': [(14, 9), (15, 9)], 'row': 456, 'col': 160, 'width': 333, 'height': 333}, 'LC82201072015017LGN00': {'chunks': [(8, 13), (8, 14)], 'row': 15

In [18]:
output = json.dumps(chunk_params)

In [19]:
with open(os.path.join(base_dir, "chunk_windows.json"), "w+") as out_file:
    out_file.write(output)

In [20]:
with open(os.path.join(base_dir, "chunk_windows.json"), "r") as test:
    testjson = json.load(test)

In [21]:
print(testjson)

{'LC80311222014338LGN00': {'chunks': [[6, 1], [6, 2], [7, 1], [7, 2]], 'row': 317, 'col': 498, 'width': 333, 'height': 333}, 'LC80631112014002LGN00': {'chunks': [[6, 6], [6, 7]], 'row': 165, 'col': 272, 'width': 333, 'height': 333}, 'LC81041072013303LGN00': {'chunks': [[6, 14], [7, 14]], 'row': 282, 'col': 20, 'width': 333, 'height': 334}, 'LC81431082013352LGN00': {'chunks': [[3, 6], [4, 6]], 'row': 508, 'col': 64, 'width': 333, 'height': 334}, 'LC81681112014002LGN00': {'chunks': [[6, 2], [6, 3], [7, 2], [7, 3]], 'row': 391, 'col': 318, 'width': 334, 'height': 334}, 'LC82071182013336LGN00': {'chunks': [[7, 8], [7, 9], [8, 8], [8, 9]], 'row': 488, 'col': 482, 'width': 334, 'height': 334}, 'LC82091172014001LGN00': {'chunks': [[15, 6], [15, 7], [16, 6], [16, 7]], 'row': 296, 'col': 505, 'width': 333, 'height': 333}, 'LC82131132013362LGN00': {'chunks': [[14, 9], [15, 9]], 'row': 456, 'col': 160, 'width': 333, 'height': 333}, 'LC82201072015017LGN00': {'chunks': [[8, 13], [8, 14]], 'row': 15

In [22]:
testjson

{'LC80311222014338LGN00': {'chunks': [[6, 1], [6, 2], [7, 1], [7, 2]],
  'row': 317,
  'col': 498,
  'width': 333,
  'height': 333},
 'LC80631112014002LGN00': {'chunks': [[6, 6], [6, 7]],
  'row': 165,
  'col': 272,
  'width': 333,
  'height': 333},
 'LC81041072013303LGN00': {'chunks': [[6, 14], [7, 14]],
  'row': 282,
  'col': 20,
  'width': 333,
  'height': 334},
 'LC81431082013352LGN00': {'chunks': [[3, 6], [4, 6]],
  'row': 508,
  'col': 64,
  'width': 333,
  'height': 334},
 'LC81681112014002LGN00': {'chunks': [[6, 2], [6, 3], [7, 2], [7, 3]],
  'row': 391,
  'col': 318,
  'width': 334,
  'height': 334},
 'LC82071182013336LGN00': {'chunks': [[7, 8], [7, 9], [8, 8], [8, 9]],
  'row': 488,
  'col': 482,
  'width': 334,
  'height': 334},
 'LC82091172014001LGN00': {'chunks': [[15, 6], [15, 7], [16, 6], [16, 7]],
  'row': 296,
  'col': 505,
  'width': 333,
  'height': 333},
 'LC82131132013362LGN00': {'chunks': [[14, 9], [15, 9]],
  'row': 456,
  'col': 160,
  'width': 333,
  'height': 